In [1]:
!pip install -r requirements.txt

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install -e .

Obtaining file:///tf/notebooks/EPT/1_taxonomist
  Attempting uninstall: taxonomist
    Found existing installation: taxonomist 0.0.3
    Can't uninstall 'taxonomist'. No files were found to uninstall.
  Running setup.py develop for taxonomist
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
!python scripts/preprocessing/process_biodiscover.py \
    --csv_path ../EPT-DL.csv \
    --out_folder data/processed/ept

In [3]:
import pandas as pd
df_csv = pd.read_csv("../EPT-DL.csv")
print(f"Number of classes: {len(df_csv['Final_species_label'].unique())}")
for label in df_csv['Final_species_label'].unique():
    print(label)

Number of classes: 14
Goera pilosa
Potamophylax rotundipennis
Baetis vernus
Baetis rhodani
Caenis horaria
Limnephilus lunatus
Chaetopteryx villosa
Amphinemura standfussi
Anabolia nervosa
Ephemera danica
Athripsodes cinereus
Mystacides azureus
Halesus radiatus
Nemoura cinerea


In [ ]:
!python scripts/01_train_test_split.py \
    --csv_path 'data/processed/ept/01_EPT_processed.csv' \
    --target_col 'Final_species_label' \
    --group_col 'individual' \
    --n_splits 5 \
    --random_state 42 \
    --out_folder 'data/processed/ept' \
    --shuffle 'False'

In [4]:
df = pd.read_csv("data/processed/ept/01_EPT_processed_5splits_Final_species_label.csv")
df.head()

,Unnamed: 0,ID,Sample Name/Number,Species Name,Image File Name,Other Notes,Max Feret Diameter,Perimeter,Area,Holes,...,Repeated Sample_DNA_repeat,Final_label,Final_species_label,Final_label_OTU,individual,0,1,2,3,4
0,0,4_F12,4_F12_1,PMR_ExStream22_Main_EPT,1_4_F12_1_2023_12_14-09-51-53-989.PNG,C25_T_mit,352,923,25295,19,...,NaN,Goera pilosa,Goera pilosa,>OTU_2,4_F12,train,train,train,test,train
1,1,4_F12,4_F12_1,PMR_ExStream22_Main_EPT,2_4_F12_1_2023_12_14-09-51-54-000.PNG,C25_T_mit,367,1034,28245,168,...,NaN,Goera pilosa,Goera pilosa,>OTU_2,4_F12,train,train,train,test,train
2,2,4_F12,4_F12_1,PMR_ExStream22_Main_EPT,1_4_F12_1_2023_12_14-09-51-54-013.PNG,C25_T_mit,400,1093,28301,21,...,NaN,Goera pilosa,Goera pilosa,>OTU_2,4_F12,train,train,train,test,train
3,3,4_F12,4_F12_1,PMR_ExStream22_Main_EPT,2_4_F12_1_2023_12_14-09-51-54-025.PNG,C25_T_mit,380,999,29520,43,...,NaN,Goera pilosa,Goera pilosa,>OTU_2,4_F12,train,train,train,test,train
4,4,4_F12,4_F12_2,PMR_ExStream22_Main_EPT,1_4_F12_2_2023_12_14-09-52-19-798.PNG,C25_T_mit,239,645,16395,1,...,NaN,Goera pilosa,Goera pilosa,>OTU_2,4_F12,train,train,train,test,train


In [5]:
import numpy as np
import pandas as pd

def even_pick(s, n):
    if len(s) <= n:
        return s
    else:
        indices = np.linspace(0, len(s)-1, num=n, dtype=int)
        return s.iloc[indices]
    
csv_list=['data/processed/ept/01_EPT_processed_5splits_Final_species_label.csv']
for num_img in [15, 25, 50, 100, 200, 500]:
    outlist = []
    for individual in df['individual'].unique():
        outlist.append(even_pick(df.query("individual == @individual"), num_img))

    df_num_img = pd.concat(outlist)
    #df_num_img.to_csv(f'data/processed/ept/01_EPT_processed_5splits_Final_species_label_{str(num_img)}.csv')
    csv_list.append(f'data/processed/ept/01_EPT_processed_5splits_Final_species_label_{str(num_img)}.csv')

In [6]:
for csv in csv_list:
    print(f'06_ept-14_img{csv[-7:-4]}_test_run')

06_ept-14_imgbel_test_run
06_ept-14_img_15_test_run
06_ept-14_img_25_test_run
06_ept-14_img_50_test_run
06_ept-14_img100_test_run
06_ept-14_img200_test_run
06_ept-14_img500_test_run


In [ ]:
df_num_img.groupby("individual").count().sort_values("Image Path", ascending=False)

In [ ]:
!python scripts/preprocessing/test_dataset_config.py \
    --data_folder "../0_images/" \
    --dataset_config "conf/user_datasets.py" \
    --dataset_name "biodiscover" \
    --csv_path "data/processed/ept/01_EPT_processed_5splits_Final_species_label.csv" \
    --label "Final_species_label" \
    --fold 0

In [7]:
import wandb
wandb.login() #581221dff0063a897a4eff9d285fa871a2c8e7f2

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: philipp-rehsen (uni-due). Use `wandb login --relogin` to force relogin


True

In [ ]:
# Only use for testing. Use all folds for actual results
!python scripts/02_train.py \
    --data_folder "../0_images/" \
    --dataset_config "conf/user_datasets.py" \
    --dataset_name "biodiscover" \
    --csv_path "data/processed/ept/01_EPT_processed_5splits_Final_species_label.csv" \
    --label "Final_species_label" \
    --fold {fold} \
    --class_map "data/processed/class_map_location.txt" \
    --imsize 224 \
    --batch_size 256 \
    --aug 'trivialaugment' \
    --load_to_memory 'False' \
    --model 'efficientnet_b0' \
    --opt 'adamw' \
    --max_epochs 10 \
    --min_epochs 0 \
    --early_stopping 'True' \
    --early_stopping_patience 50 \
    --criterion 'cross-entropy' \
    --lr 0.004 \
    --auto_lr 'True' \
    --log_dir 'tests' \
    --out_folder 'outputs' \
    --out_prefix '03_ept_run' \
    --deterministic 'True' \
    --precision '16-mixed'

In [8]:
# Only used for testing different number of images per specimen for all folds
csv_list = ['data/processed/ept/01_EPT_processed_5splits_Final_species_label_500.csv']
for csv in csv_list:
    for lr in [0.0005, 0.0001, 0.00005]:
        for fold in range(5):
            !python scripts/02_train.py \
                --data_folder "../0_images/" \
                --dataset_config "conf/user_datasets.py" \
                --dataset_name "biodiscover" \
                --csv_path {csv} \
                --label "Final_species_label" \
                --fold {fold} \
                --class_map "data/processed/class_map_location.txt" \
                --imsize 224 \
                --batch_size 256 \
                --aug 'trivialaugment' \
                --load_to_memory 'False' \
                --model 'efficientnet_b0' \
                --opt 'adamw' \
                --max_epochs 25 \
                --min_epochs 0 \
                --early_stopping 'False' \
                --early_stopping_patience 50 \
                --criterion 'cross-entropy' \
                --lr {lr} \
                --auto_lr 'False' \
                --log_dir 'ept-14-img_num_test_II' \
                --out_folder 'outputs' \
                --out_prefix f'07_ept-14_img{csv[-7:-4]}_lr{lr}_test_run' \
                --deterministic 'True' \
                --precision '16-mixed'

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b0.ra_in1k)
INFO:timm.models._hub:[timm/efficientnet_b0.ra_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
None
wandb: Currently logged in as: philipp-rehsen (uni-due). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.3
wandb: Run data is saved locally in ./wandb/run-20240704_143455-240704-1434-b4fc
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run f07_ept-14_img500_lr0.0

Epoch 1: 100%|████████████████████| 346/346 [02:04<00:00,  2.78it/s, v_num=b4fc]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  60%|██████████▊       | 52/87 [00:18<00:12,  2.84it/s]


Epoch 2: 100%|████████████████████| 346/346 [02:03<00:00,  2.81it/s, v_num=b4fc]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  67%|████████████      | 58/87 [00:21<00:10,  2.67it/s]


Epoch 3: 100%|████████████████████| 346/346 [02:02<00:00,  2.82it/s, v_num=b4fc]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  74%|█████████████▏    | 64/87 [00:21<00:07,  2.93it/s]


Epoch 4: 100%|████████████████████| 346/346 [02:03<00:00,  2.80it/s, v_num=b4fc]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  80%|██████████████▍   | 70/87 [00:26<00:06,  2.67it/s]


...

Epoch 18: 100%|███████████████████| 346/346 [02:15<00:00,  2.54it/s, v_num=cfe7]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  14%|██▌               | 12/85 [00:04<00:26,  2.73it/s]


Epoch 19: 100%|███████████████████| 346/346 [02:15<00:00,  2.55it/s, v_num=cfe7]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  24%|████▏             | 20/85 [00:07<00:25,  2.59it/s]


Epoch 20: 100%|███████████████████| 346/346 [02:15<00:00,  2.55it/s, v_num=cfe7]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  33%|█████▉            | 28/85 [00:10<00:21,  2.60it/s]


Epoch 21:  23%|████▋               | 81/346 [00:34<01:51,  2.37it/s, v_num=cfe7]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# Only used for testing different learning rates
for lr in [0.5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]:
    !python scripts/02_train.py \
        --data_folder "../0_images/" \
        --dataset_config "conf/user_datasets.py" \
        --dataset_name "biodiscover" \
        --csv_path "data/processed/ept/01_EPT_processed_5splits_Final_species_label.csv" \
        --label "Final_species_label" \
        --fold 0 \
        --class_map "data/processed/class_map_location.txt" \
        --imsize 224 \
        --batch_size 256 \
        --aug 'aug-01' \
        --load_to_memory 'False' \
        --model 'efficientnet_b0' \
        --opt 'adamw' \
        --max_epochs 10 \
        --min_epochs 0 \
        --early_stopping 'False' \
        --early_stopping_patience 50 \
        --criterion 'cross-entropy' \
        --lr {lr} \
        --auto_lr 'False' \
        --log_dir 'ept-lr-testII-aug-01' \
        --out_folder 'outputs' \
        --out_prefix '00_lr-testII_aug-01_ept_run' \
        --deterministic 'True' \
        --precision '16-mixed'

In [ ]:
for fold in range(5):
    !python scripts/02_train.py \
        --data_folder "../0_images/" \
        --dataset_config "conf/user_datasets.py" \
        --dataset_name "biodiscover" \
        --csv_path "data/processed/ept/01_EPT_processed_5splits_Final_species_label.csv" \
        --label "Final_species_label" \
        --fold {fold} \
        --class_map "data/processed/class_map_location.txt" \
        --imsize 224 \
        --batch_size 256 \
        --aug 'trivialaugment' \
        --load_to_memory 'False' \
        --model 'efficientnet_b0' \
        --opt 'adamw' \
        --max_epochs 100 \
        --min_epochs 0 \
        --early_stopping 'False' \
        --early_stopping_patience 50 \
        --criterion 'cross-entropy' \
        --lr 0.0001 \
        --auto_lr 'False' \
        --log_dir 'ept-14' \
        --out_folder 'outputs' \
        --out_prefix '05_preliminary_ept_run' \
        --deterministic 'True' \
        --precision '16-mixed'

In [ ]:
# This just sets the checkpoint as the first (best) model in the directory above, as the unique identifier is always different.
!python scripts/03_predict.py \
    --data_folder "../0_images/" \
    --dataset_config "conf/user_datasets.py" \
    --dataset_name "biodiscover" \
    --csv_path "data/processed/ept/01_EPT_processed_5splits_Final_species_label.csv" \
    --label "Final_species_label" \
    --fold 4 \
    --class_map "data/processed/class_map_location.txt" \
    --imsize 224 \
    --batch_size 1024 \
    --aug 'none' \
    --out_folder 'outputs' \
    --tta 'False' \
    --out_prefix '' \
    --ckpt_path 'outputs/biodiscover/05_preliminary_ept_run_efficientnet_b0/f4/05_preliminary_ept_run_efficientnet_b0_f4_240601-1251-111a_epoch00_val-loss0.78.ckpt'

In [ ]:
!python scripts/04_group_predictions.py \
    --predictions "outputs/biodiscover/aug_resnet18/f0/predictions/biodiscover_none/_aug_resnet18_f0_240524-1427-0065_epoch50_val-loss1.62_last_none.csv" \
    --reference_csv "data/processed/ept/01_EPT_processed_5splits_Final_species_label.csv" \
    --reference_target "Final_species_label" \
    --fold 0 \
    --reference_group "individual" \
    --agg_func "mode"